In [ ]:
# Structural Design supported by Machine Learning
# Original code developed by Vahid Moosavi (sevamoo@gmail.com) and adapted by Pierluigi D'Acunto (pierluigi.dacunto@tum.de)

#If you use the scripts please reference the official GitHub repository:
#@Misc{sdml2021,
#author = {D'Acunto, Pierluigi and Ohbrock, Patrick Ole and Saldana Ochoa, Karla and Moosavi, Vahid},
#title = {{SDML: Structural Design supported by Machine Learning}},
#year = {2021},
#url = {https://github.com/pierluigidacunto/SDML},
#}

# Machine Learning-Assisted Computational Structural Design

## 0 Environment

In [ ]:
!pip install numpy
!pip install pandas
!pip install scipy
!pip install matplotlib
!pip install numexpr

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import sompylib.sompy as SOM
import math
import pathlib
import os
import matplotlib.pyplot as plt

## 1 Reading CEM data

Option 1: Zurich bridge

In [ ]:
# data_file = "CEM/Data/240806_data_00_CEM.csv"
# graph_file = "CEM/Data/240806_data_00_branch-node_CEM.csv"

Option 2: simple 2D bridge

In [ ]:
data_file = "CEM/Data/240818_data_simple_CEM.csv"
graph_file = "CEM/Data/240818_data_simple_branch-node_CEM.csv"

In [ ]:
# Read CSV file and create DataFrame
root_path = pathlib.Path().absolute()

data = pd.read_csv(
    os.path.join(
        root_path,
        data_file),
    header=None)

data = data.dropna()
data.head(10)  # show first 10 entries of DataFrame

In [ ]:
# Import branch-node matrix
graph_edges = pd.read_csv(
    os.path.join(
        root_path,
        graph_file),
    header=None)

graph_edges.head(10)

## 1.1 Data description

### Data CSV Option 1: Zurich Bridge

| DESCRIPTION | VARIABLE NAME | DIMENSION | SOURCE |
| --- | --- | --- | --- |
| deviation force at main cable midspan | cable_center_deviation_force | 1 | input |
| deviation force at deck midspan | deck_center_deviation_force | 1 | input |
| force in hangars | hangar_force | 1 | input |
| force in deviations inside deck | deck_deviation_force | 1 | input |
| force in deviations between main cables | cable_deviation_force | 1 | input |
| force in support towers | tower_force | 1 | input |
| force in brace between support towers | tower_brace_force | 1 | input |
| rise of the deck | deck_rise | 1 | input
| vertical distance between the deck and main cables at midspan | cable_deck_midspan_distance | 1 | input |
| offset of tower base location in x direction/perpendicular to the river | tower_base_x_offset | 1 | input |
| offset of tower base location in y direction/parallel | tower_base_y_offset | 1 | input |
| offset of origin nodes at midspan to create a twist in the bridge | twist_offset | 1 | input |
| number of nodes | nN | 1 | output |
| number of trail edges | nT | 1 | output |
| number of deviation edges | nD | 1 | output |
| load X | loadX | 1 | input |
| load Y | loadY | 1 | input |
| load Z | loadZ | 1 | input |
| nodes position X | posX | nN | output |
| nodes position Y | posX | nN | output |
| nodes position Z | posX | nN | output |
| trail lenghts | traLen | nT | output |
| trail magnitudes | traMag | nT | output |
| deviation lenghts | devLen | nD | output |
| deviation magnitudes | devMag | nD | input |
| total load path | load_path | 1 | output |
| highest inclination of the deck in degrees | maximum deck inclination | 1 | output |

In [ ]:
label_columns_fix_head = [
    "cable_center_deviation_force",
    "deck_center_deviation_force",
    "hangar_force",
    "deck_deviation_force",
    "cable_deviation_force",
    "tower_force",
    "tower_brace_force",
    "deck_rise",
    "cable_deck_midspan_distance",
    "tower_base_x_offset",
    "tower_base_y_offset",
    "twist_offset",
    "nN",
    "nT",
    "nD",
    "loadX",
    "loadY",
    "loadZ"
]

label_columns_fix_tail = [
    "load_path",
    "maximum_deck_inclination"
]

nX = len(label_columns_fix_head)    # number of fixed entries
print('Numer of fixed entries is {0}'.format(nX))

### Data CSV Option 2: Simple 2D Bridge

| DESCRIPTION | VARIABLE NAME | DIMENSION | SOURCE |
| --- | --- | --- | --- |
| number of nodes | nN | 1 | fixed |
| number of trail edges | nT | 1 | fixed |
| number of deviation edges | nD | 1 | fixed |
| load X | loadX | 1 | input |
| load Y | loadY | 1 | input |
| load Z | loadZ | 1 | input |
| nodes position X | posX | nN | output |
| nodes position Y | posX | nN | output |
| nodes position Z | posX | nN | output |
| trail lenghts | traLen | nT | output |
| trail magnitudes | traMag | nT | output |
| deviation lenghts | devLen | nD | output |
| deviation magnitudes | devMag | nD | input |
| total load path | load_path | 1 | output |
| highest inclination of the deck in degrees | maximum deck inclination | 1 | output |

In [ ]:
label_columns_fix_head = [
    "nN",
    "nT",
    "nD",
    "loadX",
    "loadY",
    "loadZ"
]

label_columns_fix_tail = [
]

nX = len(label_columns_fix_head)    # number of fixed entries
print('Numer of fixed entries is {0}'.format(nX))

### Graph CSV
| DESCRPTION | VARIABLE | DIMENSION | INDEX | SOURCE |
| --- | --- | --- | --- | --- |
| edge start | start | nT + nD | even (0, 2, 4, ..., (nT + nD) * 2) | input |
| edge end | end | nT + nD | odd (1, 3, 5, ..., (nT + nD) * 2 + 1) | input |

Generate labels for columns in DataFrame

In [ ]:
nN = int(data.iloc[0][nX - 6])
nT = int(data.iloc[0][nX - 5])
nD = int(data.iloc[0][nX - 4])

print('Number of Nodes = {0}\nNumber of Trail Edges = {1}\nNumber of Deviation Edges = {2}'.format(nN, nT, nD))

In [ ]:
label_columns = label_columns_fix_head[:]
label_columns += ["posX_" + str(i) for i in range(nN)]
label_columns += ["posY_" + str(i) for i in range(nN)]
label_columns += ["posZ_" + str(i) for i in range(nN)]    
label_columns += ["traLen_" + str(i) for i in range(nT)]
label_columns += ["traMag_" + str(i) for i in range(nT)]
label_columns += ["devLen_" + str(i) for i in range(nD)]
label_columns += ["devMag_" + str(i) for i in range(nD)]
label_columns += label_columns_fix_tail

data.columns = label_columns
data.head(10)

## 1.2 Graph Visualiztion

In [ ]:
nE = nT + nD

edges = [(int(graph_edges.iloc[0][i * 2]), int(graph_edges.iloc[0][i * 2 + 1])) for i in range(nE)]

for i in range(min(10, len(edges))):
    print('edge {0}: start node {1}, end node {2}'.format(i, edges[i][0], edges[i][1]))
print('...')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def show_sample(
    data_frame, 
    sample_index, 
    view='3D-30',
    ax = None, 
    show_title = True,
    title = None,
    title_size=25):
    # array of node xyz coordinates
    p0 = nX
    p1 = p0 + 3*nN
    pos_xyz = data_frame.iloc[sample_index][p0:p1]
    
    pos_x = pos_xyz[: nN]
    pos_y = pos_xyz[nN : 2*nN]
    pos_z = pos_xyz[2*nN:]
    coords = np.array([pos_x, pos_y, pos_z]).T  
    
    # array of trail edges    
    p0 = p1
    p1 = p0 + nT
    trail_len = data_frame.iloc[sample_index][p0:p1]
    
    p0 = p1
    p1 = p0 + nT
    trail_mag = data_frame.iloc[sample_index][p0:p1]
    
    # array of deviation edges    
    p0 = p1
    p1 = p0 + nD
    dev_len = data_frame.iloc[sample_index][p0:p1]
    
    p0 = p1
    p1 = p0 + nD
    dev_mag = data_frame.iloc[sample_index][p0:p1]

    edges_mag = np.concatenate((trail_mag, dev_mag), axis = None)
    
    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
    
    if show_title:
        if title is not None:
            ax.set_title(title, size=title_size)
        else:
            ax.set_title('sample {0}'.format(str(data_frame.index[sample_index])), size=title_size)
    ax.set_aspect('auto')
    ax.axis('off')
    
    # Set view
    ax.set_proj_type('ortho')
    
    if view == '2D-XY':
        ax.view_init(elev=90, azim=-90) # 2D-XY
    elif view == '2D-XZ':
        ax.view_init(elev=0, azim=-90) # 2D-XZ
    elif view == '2D-YZ':
        ax.view_init(elev=0, azim=0) # 2D-YZ
    elif view == '3D-45':
        ax.view_init(elev=45, azim=45) # 3D-45
    else:
        ax.view_init(elev=30, azim=60) # 3D-30
    
    # make xyz scale equal
    vrange=coords.max(axis=0) - coords.min(axis=0)
    vrange=vrange.max()/2
    vmean=coords.mean(axis=0)
    ax.set_xlim(vmean[0]-vrange,vmean[0]+vrange)
    ax.set_ylim(vmean[1]-vrange,vmean[1]+vrange)
    ax.set_zlim(vmean[2]-vrange,vmean[2]+vrange)
    
    for edge, mag in zip(edges, edges_mag):
        line_color = (5/256,120/256,190/256) if np.sign(mag) < 0 else (200/256,20/256,20/256)
        line_width = 0.3 + np.abs(mag/40)
             
        ax.plot(coords[edge,0], 
                coords[edge,1],
                coords[edge,2], 
                color = line_color, 
                linewidth = line_width, 
                antialiased=True, 
                alpha = 1.0)

In [ ]:
show_sample(data, 1, '2D-XZ')
plt.show()

## 2. Compute Feature Vectors

We pre-calculate high-order statistics (Min, Max, Mean, Standard Deviation, Skewness, Kurtosis) per type (Floor Height, Node, Trail, Deviation)

In [ ]:
from numpy import mean
from numpy import std
from scipy.stats import skew
from scipy.stats import kurtosis

# Convert initial DataFrame to Numpy 2D array
src_data = data.values[:]

# Initialize list of list where initial variable features are replaced by their corresponding statistics
data_arr2d = []

# Iterate through each row of the array (i.e. each row of the DataFrame)
for i in range(len(src_data)): 
    src_row = src_data[i]
    # Calculate statistics per feature    
    p0 = nX
    p1 = p0 + nN
    posX_arr = src_row[p0:p1]
    posX_min = posX_arr.min()
    posX_max = posX_arr.max()
    posX_mean = mean(posX_arr)
    posX_std = std(posX_arr)
    posX_skew = skew(posX_arr)
    posX_kurt = kurtosis(posX_arr)

    p0 = p1
    p1 = p0 + nN
    posY_arr = src_row[p0:p1]
    posY_min = posY_arr.min()
    posY_max = posY_arr.max()
    posY_mean = mean(posY_arr)
    posY_std = std(posY_arr)
    posY_skew = skew(posY_arr)
    posY_kurt = kurtosis(posY_arr)    

    p0 = p1
    p1 = p0 + nN
    posZ_arr = src_row[p0:p1]
    posZ_min = posZ_arr.min()
    posZ_max = posZ_arr.max()
    posZ_mean = mean(posZ_arr)
    posZ_std = std(posZ_arr)
    posZ_skew = skew(posZ_arr)
    posZ_kurt = kurtosis(posZ_arr)

    p0 = p1
    p1 = p0 + nT
    traLen_arr = src_row[p0:p1]
    traLen_min = traLen_arr.min()
    traLen_max = traLen_arr.max()
    traLen_mean = mean(traLen_arr)
    traLen_std = std(traLen_arr)
    traLen_skew = skew(traLen_arr)
    traLen_kurt = kurtosis(traLen_arr) 
           
    p0 = p1
    p1 = p0 + nT
    traMag_arr = src_row[p0:p1]
    traMag_min = traMag_arr.min()
    traMag_max = traMag_arr.max()
    traMag_mean = mean(traMag_arr)
    traMag_std = std(traMag_arr)
    traMag_skew = skew(traMag_arr)
    traMag_kurt = kurtosis(traMag_arr)
    
    p0 = p1
    p1 = p0 + nD
    devLen_arr = src_row[p0:p1]
    devLen_min = devLen_arr.min()
    devLen_max = devLen_arr.max()
    devLen_mean = mean(devLen_arr)
    devLen_std = std(devLen_arr)
    devLen_skew = skew(devLen_arr)
    devLen_kurt = kurtosis(devLen_arr) 
           
    p0 = p1
    p1 = p0 + nD
    devMag_arr = src_row[p0:p1]
    devMag_min = devMag_arr.min()
    devMag_max = devMag_arr.max()
    devMag_mean = mean(devMag_arr)
    devMag_std = std(devMag_arr)
    devMag_skew = skew(devMag_arr)
    devMag_kurt = kurtosis(devMag_arr)

    # Assemble flattened features in an array
    data_arr = [
                posX_min, posX_max, posX_mean, posX_std, posX_skew, posX_kurt,
                posY_min, posY_max, posY_mean, posY_std, posY_skew, posY_kurt,
                posZ_min, posZ_max, posZ_mean, posZ_std, posZ_skew, posZ_kurt,
                traLen_min, traLen_max, traLen_mean, traLen_std, traLen_skew, traLen_kurt,
                traMag_min, traMag_max, traMag_mean, traMag_std, traMag_skew, traMag_kurt,
                devLen_min, devLen_max, devLen_mean, devLen_std, devLen_skew, devLen_kurt,
                devMag_min, devMag_max, devMag_mean, devMag_std, devMag_skew, devMag_kurt,
                ]
    
    data_arr2d.append(data_arr)

In [ ]:
# Convert list of list into DataFrame
data_hos = pd.DataFrame.from_records(data_arr2d)
data_hos.index = data.index

label_columns = [
                "posX_min", "posX_max", "posX_mean", "posX_std", "posX_skew", "posX_kurt",
                "posY_min", "posY_max", "posY_mean", "posY_std", "posY_skew", "posY_kurt",
                "posZ_min", "posZ_max", "posZ_mean", "posZ_std", "posZ_skew", "posZ_kurt",
                "traLen_min", "traLen_max", "traLen_mean", "traLen_std", "traLen_skew", "traLen_kurt",
                "traMag_min", "traMag_max", "traMag_mean", "traMag_std", "traMag_skew", "traMag_kurt",
                "devLen_min", "devLen_max", "devLen_mean", "devLen_std", "devLen_skew", "devLen_kurt",
                "devMag_min", "devMag_max", "devMag_mean", "devMag_std", "devMag_skew", "devMag_kurt",
                ]

data_hos.columns = label_columns
data_hos.head(10)

In [ ]:
# merge the HOS data with other fixed-length data in the source data frame

data_hos = pd.concat(
    [
        data[label_columns_fix_head], 
        data_hos, 
        data[label_columns_fix_tail]
    ],
    axis=1)

data_hos.head(10)

## 2.1 Filter Data

In [ ]:
# Filter DataFrame 1

traMag_max_mask = abs(data_hos['traLen_max']) < abs(data_hos['traLen_max']).quantile(0.9)
data_hos_filter = data_hos[traMag_max_mask].copy()

data_hos_filter.shape

In [ ]:
# Filter DataFrame 2

devLen_min_mask = abs(data_hos_filter['devLen_min']) > abs(data_hos_filter['devLen_min']).quantile(.1)
data_hos_filter = data_hos_filter[devLen_min_mask].copy()

data_hos_filter.shape

For Zurich Bridge only

In [ ]:
# # Filter DataFrame 3
# totLP_mask = abs(data_hos_filter['load_path']) < abs(data_hos_filter['load_path']).quantile(.9)
# data_hos_filter = data_hos_filter[totLP_mask].copy()

# data_hos_filter.shape

In [ ]:
# Update initial DataFrame in compliance with previous filtering on flat DataFrame
data_filter = data[data.index.isin(data_hos_filter.index)]
data_filter.head(10)

## 2.2 Feature Vector for SOM

In [ ]:
# Select features for SOM training

SOM_features = [
#     'maximum_deck_inclination',
#     'load_path',
#     'posX_min',
#     'posX_max',    
    'posX_mean', 
    'posX_std', 
    'posX_skew', 
    'posX_kurt', 
#     'posY_min',
#     'posY_max',    
#     'posY_mean', 
#     'posY_std', 
#     'posY_skew', 
#     'posY_kurt',
#     'posZ_min',
#     'posZ_max',  
    'posZ_mean',
    'posZ_std', 
    'posZ_skew', 
    'posZ_kurt',
#     'traLen_min',    
#     'traLen_max',    
    'traLen_mean',
    'traLen_std', 
    'traLen_skew', 
#     'traLen_kurt', 
#     'traMag_min',    
#     'traMag_max',    
#     'traMag_mean',
#     'traMag_std', 
#     'traMag_skew', 
#     'traMag_kurt',
#     'devLen_min',    
#     'devLen_max',    
#     'devLen_mean',
#     'devLen_std', 
#     'devLen_skew', 
#     'devLen_kurt', 
#     'devMag_min',    
#     'devMag_max',    
    'devMag_mean',
    'devMag_std', 
    'devMag_skew', 
#     'devMag_kurt'
    ]

# Create matrix of selected features
features_arr = data_hos_filter[SOM_features].values[:].astype(float)
features_arr.shape

## 3 Train the SOM

In [ ]:
# Calculate SOM

ncols = 10    # define number of SOM-nodes in X directions
nrows = 10    # define number of SOM-nodes in Y directions
mapsize = (nrows, ncols)

som = SOM.SOM(
    '',
    features_arr,
    mapsize = mapsize,
    norm_method = 'var',
    initmethod = 'pca')

# feature name for plotting
som.compname = [SOM_features]
som.train(n_job = 1, shared_memory = 'no',verbose = 'on')

### 3.1 Best matching units (BMUs)

In [ ]:
# Get distance/s and Best Matching Unit (BMU)/s indices for each sample
dists, bmus = som.find_K_nodes(som.data_raw, K=1)

for i in range(min(10, len(bmus))):
    print("sample {0}'s BMU is {1}".format(i, bmus[i, 0]))

In [ ]:
# Get representative samples per SOM-nodes (samples that are closer to their corresponding BMUs)
cell_sample_index = []

for i in range(som.nnodes):
    # Get all samples per SOM-node
    samples_index = np.argwhere(bmus==i)[:,0]

    if len(samples_index) > 0:
        # Sort samples according to distance within each BMU cluster
        samples_dist = dists[samples_index][:,0]
        indices = np.argsort(samples_dist, axis=-1)
        cell_sample_index.append(samples_index[indices[0]]) # representative sample for SOM-node
    else:
        cell_sample_index.append(-1)
        
for i in range(min(10, len(cell_sample_index))):
    print('cell {0}\'s representative sample is {1}'.format(i, cell_sample_index[i]))
print('...')

### 3.2 Visualize SOM as a grid

In [ ]:
def show_som(som, cell_sample_index, nrows, ncols):
    fig = plt.figure(figsize=(2*ncols,2*nrows))
    gs = fig.add_gridspec(nrows, ncols)
    gs.update(wspace=0.0, hspace=0.0)

    for i in range(som.nnodes):
        row = i // ncols
        col = i % ncols

        f = cell_sample_index[i]    # representative sample f

        if f > -1:              
            ax = fig.add_subplot(gs[row,col], projection='3d')
            show_sample(
                data_filter,
                f, 
                '2D-XZ', 
                ax, 
                show_title = True, 
                title = 'cell {0}'.format(i),
                title_size = 8)
    plt.show()

In [ ]:
show_som(som, cell_sample_index, nrows, ncols)

In [ ]:
if features_arr.shape[1] > 100:
    pass
else:
    som.view_map(text_size=9)

### 3.3 Visualize SOM cells

In [ ]:
def show_node(cell_index, *col_names):
    # Get all samples in selected SOM cell
    indices = np.argwhere(bmus==cell_index)[:,0]
    
    # Number of samples in selected SOM cell
    K = len(indices) 
    print('number samples: ', K)
    
    ncols = min(5, K)
    nrows = max(1, int(math.ceil(K / ncols)))
        
    if K >= 1:
        fig = plt.figure(figsize=(4*ncols,4*nrows))
        gs = fig.add_gridspec(nrows, ncols)
        gs.update(wspace=0.0, hspace=0.0)

        for i in range(K):
            row = i // ncols
            col = i % ncols
            f = indices[i]    # representative sample f

            if f > -1:              
                ax = fig.add_subplot(gs[row,col], projection='3d')
                
                title = []
                if len(col_names)>0:
                    title = ['{0}: {1:.2f}'.format(col, data_hos_filter.iloc[f][col]) for col in col_names]
                
                title = '\n'.join(['sample {0}'.format(data_filter.index[f])] + title)
                    
                show_sample(
                    data_filter,
                    f, 
                    '2D-XZ', 
                    ax, 
                    show_title = title,
                    title = title, title_size=8)
                
        plt.show()
    else:
        print ('not enough data for this node')

In [ ]:
show_node(0, 'devLen_mean', 'traLen_mean')

In [ ]:
show_node(7, 'devLen_mean', 'traLen_mean')

In [ ]:
show_node(35, 'devLen_mean', 'traLen_mean')

In [ ]:
show_node(42, 'devLen_mean', 'traLen_mean')

In [ ]:
show_node(80, 'devLen_mean', 'traLen_mean')